In [1]:
using Revise

In [6]:
includet("tvp-var.jl")

In [7]:
step(1, .4, 0.3, .001, 0.02, .09, SVparams(0.05, 0.7))

0.2404662811790583

In [37]:
Β =  generate_coeffs([.2, .2, .2], [.1, .8, .1], [.0001, .0001, .0001], [.02, .005, .01] , 8)
round.(Β, digits=4)

3×8 Matrix{Float64}:
 0.202   0.2079  0.2205  0.2204  0.2136  0.2176  0.2215  0.202
 0.0635  0.0952  0.117   0.1732  0.0934  0.1122  0.1099  0.0613
 0.2015  0.2091  0.2048  0.2084  0.2155  0.2106  0.2127  0.1994

In [36]:
Β =  generate_coeffs([.2, .2, .2], [.1, .8, .1], [.0001, .0001, .0001], [.02, .005, .01] , 8)
round.(Β, digits=4)

3×8 Matrix{Float64}:
 0.3039  0.2887  0.2886  0.2785  0.276   0.286   0.2874  0.283
 0.1804  0.1104  0.113   0.0608  0.0239  0.1181  0.1162  0.1093
 0.2107  0.207   0.2228  0.2145  0.22    0.231   0.2213  0.216